In [ ]:
#https://www.ratemyimmigrationjudge.com/listing/hon-john-w-cortes-immigration-judge-new-york-varick-immigration-court/

#https://www.ratemyimmigrationjudge.com/job-listings/?search_keywords=&search_location=&search_radius=50&search_lat=0&search_lng=0

In [48]:
import csv
import pandas as pd
import googlemaps
import requests 
import urllib.request
import time
from bs4 import BeautifulSoup
import pickle

In [49]:
df2 = pickle.load(open('imm_judges.p','rb'))
df2.head()

/home/awstclair/miniconda3/envs/geopandas_env/lib/python3.6/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.9.1dev-CAPI-1.14.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


,Court,Address,Judges,Ct Admin,Street,CityZip,Phone,cz_fix,Latitude,Longitude,geometry
0,Eloy,"1705 E. Hanna Road, Suite 366\n Eloy, AZ 851...","[Cortes, John W., Gaz, Jennifer I., Habich, Pa...","Padilla, Victoria","1705 E. Hanna Road, Suite 366","Eloy, AZ 85131",520-466-3671,"[Eloy,, AZ, 85131]",32.755170,-111.553493,POINT (-111.55349 32.75517)
1,Florence,"3260 N. Pinal Parkway Avenue\n Florence, AZ ...","[Guerrero, Marni, Taylor, Bruce A.]","Toliusis, Rachel",3260 N. Pinal Parkway Avenue,"Florence, AZ 85132",520-868-3341,"[Florence,, AZ, 85132]",33.031451,-111.387343,POINT (-111.38734 33.03145)
2,Phoenix,"250 N. Seventh Ave., Suite 300\n Phoenix, AZ...","[Bartlemay Sr., Robert C., Cottor, Elizabeth A...","Riebs, Sathya","250 N. Seventh Ave., Suite 300","Phoenix, AZ 85007",602-640-2747,"[Phoenix,, AZ, 85007]",33.448437,-112.074142,POINT (-112.07414 33.44844)
3,Tucson,"300 West Congress, Suite 300\n Tucson, AZ 85...","[Davis, John W., DeAngelis, Kathryn L., Keenan...","Graff, Corey","300 West Congress, Suite 300","Tucson, AZ 85701",520-670-5212,"[Tucson,, AZ, 85701]",32.222876,-110.974848,POINT (-110.97485 32.22288)
4,Adelanto,Adelanto Detention Facility\n 10250 Rancho R...,"[DePowell, Bryan E., Halperin, Ravit R., Marcu...","Roberts, Valerie (Acting)","10250 Rancho Road, Ste. 201A","Adelanto, CA 92301",760-561-6500,"[Adelanto,, CA, 92301]",34.582770,-117.409215,POINT (-117.40922 34.58277)


# tester

In [2]:
url = 'https://www.ratemyimmigrationjudge.com/listing/hon-john-w-cortes-immigration-judge-new-york-varick-immigration-court/'
response = requests.get(url)
response

<Response [200]>

In [3]:
soup = BeautifulSoup(response.text,'html.parser')

In [6]:
bio_test = soup.findAll('p')

bio_test[0:1]

[<p>Then-Acting Attorney General Matthew Whitaker appointed John W. Cortes to begin hearingcases in March 2019. Judge Cortes earned a Bachelor of Arts in 1999 from the City University ofNew York at Brooklyn College and a Juris Doctor in 2002 at the New York Law School. From2003 to 2018, he served as an assistant district attorney for the Suffolk County DistrictAttorneyâ€™s Office, and from 2013 to 2018, he served as a principal assistant district attorney forthe same court. Judge Cortes is a member of the State Bar of Arizona and New York State Bar.</p>]

# Get Court pages

In [51]:
# https://www.ratemyimmigrationjudge.com/job-listings/?search_keywords=&search_location=eloy&search_radius=50&search_lat=0&search_lng=0

court_pages = []
for court in df2['Court']:
    url = 'https://www.ratemyimmigrationjudge.com/job-listings/?search_keywords=&search_location={}&search_radius=50&search_lat=0&search_lng=0'.format(court)
    court_pages.append(url)
court_pages[0:5]

['https://www.ratemyimmigrationjudge.com/job-listings/?search_keywords=&search_location=Eloy&search_radius=50&search_lat=0&search_lng=0',
 'https://www.ratemyimmigrationjudge.com/job-listings/?search_keywords=&search_location=Florence&search_radius=50&search_lat=0&search_lng=0',
 'https://www.ratemyimmigrationjudge.com/job-listings/?search_keywords=&search_location=Phoenix&search_radius=50&search_lat=0&search_lng=0',
 'https://www.ratemyimmigrationjudge.com/job-listings/?search_keywords=&search_location=Tucson&search_radius=50&search_lat=0&search_lng=0',
 'https://www.ratemyimmigrationjudge.com/job-listings/?search_keywords=&search_location=Adelanto&search_radius=50&search_lat=0&search_lng=0']

In [61]:

url = court_pages[0]
response = requests.get(url)
response
soup = BeautifulSoup(response.text,'html.parser')


# now judges by page

In [88]:
df2['Judges'].head()

0    [Cortes, John W., Gaz, Jennifer I., Habich, Pa...
1                  [Guerrero, Marni, Taylor, Bruce A.]
2    [Bartlemay Sr., Robert C., Cottor, Elizabeth A...
3    [Davis, John W., DeAngelis, Kathryn L., Keenan...
4    [DePowell, Bryan E., Halperin, Ravit R., Marcu...
Name: Judges, dtype: object

In [56]:
judge_list =[]

for judge in df2['Judges'][0]:
    judge_list.append(judge)
judge_list[0:20]

['Cortes, John W.',
 'Gaz, Jennifer I.',
 'Habich, Paul',
 'Penalosa Jr., Jose L.']

In [204]:
sample = df2['Judges'][0][3]

def judge_addy_part(judge):
    j_split = judge.split()
    res = [j.strip('.,') for j in j_split]
    name_num = len(res)
    # judge_hyphen =j_split
    if name_num == 1:
        judge_hyphen = res
    elif name_num == 2:
        judge_hyphen = res[1]+'-'+res[0]
    elif name_num == 3:
        judge_hyphen = res[1]+'-'+res[2]+'-'+res[0]
    else:
        judge_hyphen = res[2]+'-'+res[3]+'-'+res[0]+'-'+res[1]
    return judge_hyphen

judge_addy_part(df2['Judges'][0][3])


'Jose-L-Penalosa-Jr'

In [203]:
sample = df2['Judges'][0][3]

def judge_addy_list(list):
    j_list=[]
    for judge in list:
        j_split = judge.split()
        res = [j.strip('.,') for j in j_split]
        name_num = len(res)
        # judge_hyphen =j_split
        if name_num == 1:
            judge_hyphen = res
        elif name_num == 2:
            judge_hyphen = res[1]+'-'+res[0]
        elif name_num == 3:
            judge_hyphen = res[1]+'-'+res[2]+'-'+res[0]
        else:
            judge_hyphen = res[2]+'-'+res[3]+'-'+res[0]+'-'+res[1]
        j_list.append(judge_hyphen)
    return j_list

judge_addy_list(df2['Judges'][0])

['John-W-Cortes', 'Jennifer-I-Gaz', 'Paul-Habich', 'Jose-L-Penalosa-Jr']

In [198]:
judge_list_HY =[]
#https://www.ratemyimmigrationjudge.com/listing/hon-john-w-cortes-immigration-judge-new-york-varick-immigration-court/

for judge in df2['Judges'][0]:
    judge_list_HY.append(judge_addy_part(judge))
judge_list_HY
    
    

['John-W-Cortes', 'Jennifer-I-Gaz', 'Paul-Habich', 'Jose-L-Penalosa-Jr']

In [209]:
judge_addy_list(df2['Judges'][2])

['Robert-C-Bartlemay-Sr',
 'Elizabeth-A-Cottor',
 'LaMonte-S-Freerks',
 'Joseph-S-Imburgia',
 'Ken-Josephson',
 'Melissa-B-Karlen',
 'Munish-Sharda',
 'Linda-Spencer-Walters']

In [210]:
df2['Judges_Hy'] = df2['Judges'].apply(lambda x : [judge_addy_list(x) for x in df2['Judges']])

IndexError: list index out of range

In [156]:
df2['CityZip'][0].split()[0].strip(',.')

'Eloy'

In [181]:
judge_city =[]
for i in range(len(df2['Judges'])):
    for j in df2['Judges'][i]:
        hy_j = judge_addy_part(j)
        city = df2['CityZip'][i]
        try:
            city = city.split()[0].strip(',.')
        except:
            pass
        # judge_hy = judge_addy_part(j)
        judge_city.append((hy_j,city))
judge_city[0:15]

[('Cortes, John W.', 'Eloy'),
 ('Gaz, Jennifer I.', 'Eloy'),
 ('Habich, Paul', 'Eloy'),
 ('Penalosa Jr., Jose L.', 'Eloy'),
 ('Guerrero, Marni', 'Florence'),
 ('Taylor, Bruce A.', 'Florence'),
 ('Bartlemay Sr., Robert C.', 'Phoenix'),
 ('Cottor, Elizabeth A.', 'Phoenix'),
 ('Freerks, LaMonte S.', 'Phoenix'),
 ('Imburgia, Joseph S.', 'Phoenix'),
 ('Josephson, Ken', 'Phoenix'),
 ('Karlen, Melissa B.', 'Phoenix'),
 ('Sharda, Munish', 'Phoenix'),
 ('Spencer-Walters, Linda', 'Phoenix'),
 ('Davis, John W.', 'Tucson')]

# now indy bios for judges

In [64]:
url3= https://www.ratemyimmigrationjudge.com/listing/hon-print-maggard-immigration-judge-san-francisco-immigration-court/
#hon-print-maggard-immigration-judge-san-francisco-immigration-court/
response3 = requests.get(url)
response3

<Response [200]>

In [69]:
soup3 = BeautifulSoup(response3.text,'html.parser')
judge_list3 = soup3.body
